In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup
import pandas as pd
import re

## Extract

In [2]:
PRICE_RE      = re.compile(r'\$\s*\d{1,3}(?:\.\d{3})+(?:,\d+)?')
DISCOUNT_RE   = re.compile(r'(\d{1,2})\s*%\s*OFF', re.I)
SENTENCE_SPLIT_RE = re.compile(r'\.(?=\s*[A-ZÁÉÍÓÚÑ])')

In [ ]:
def extractor_mercadolibre(cantidad, buscador):
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.get('https://www.mercadolibre.com.co/')

    # Espera hasta que el input esté presente
    wait = WebDriverWait(driver, 10)
    search_box = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="cb1-edit"]')))

    # Escribe en el campo de búsqueda
    search_box.send_keys(buscador)

    # Espera hasta que el botón esté presente y haz clic
    search_button = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/header/div/div[2]/form/button')))
    search_button.click()

    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li.ui-search-layout__item')))

    # Parsear con BeautifulSoup
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # Selecciona todos los items
    items = soup.select('li.ui-search-layout__item')

    contador = 0
    productos = []

    for item in items:
        link = item.select_one('a.poly-component__title')
        driver.switch_to.new_window()
        driver.get(link["href"])
        
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.ui-pdp-container.ui-pdp-container--pdp')))
        
        # Parsear con BeautifulSoup
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        price = soup.select_one('div.ui-pdp-price__second-line')    
        cuotas = soup.select_one('div.ui-pdp-price__subtitles')
        informacion = soup.select_one('ul.ui-vpp-highlighted-specs__features-list')
        
        m_price = PRICE_RE.search(price.text) if price else None
        m_discount = DISCOUNT_RE.search(price.text) if price else None
        m_informacion = SENTENCE_SPLIT_RE.split(informacion.text) if informacion else None
        
        producto = {
            "Nombre": link.text.strip() if link else 'No name found',
            "precio": m_price.group(0) if m_price else 'No price found',
            "descuento": m_discount.group(0) if m_discount else 'No discount found',
            "cuotas": cuotas.text if cuotas else 'No cuotas found',
            "informacion": m_informacion if m_informacion else 'No informacion found'
        }

        productos.append(producto)

        contador += 1
        
        if contador >= cantidad:
            break

    driver.quit()
    return productos

In [ ]:
productos = extractor_mercadolibre(20, 'computador')

In [22]:
df = pd.DataFrame(productos)
df

,precio,descuento,cuotas,informacion,_id
0,$5.639.900,16% OFF,9 cuotas de $626.656 con 0% interés,No informacion found,68b09ec6210a34c1d0c4d861
1,$2.021.136,No discount found,9 cuotas de $224.571 con 0% interés,"[Procesador: Intel Core i5 1235U, Nombre del s...",68b09ec6210a34c1d0c4d862
2,$1.175.900,No discount found,6 cuotas de $195.983 con 0% interés,"[Capacidad de disco SSD: 256 GB, Capacidad tot...",68b09ec6210a34c1d0c4d863
3,$1.539.900,No discount found,9 cuotas de $171.100 con 0% interés,"[Procesador: Intel Core i3 1215U, Versión del ...",68b09ec6210a34c1d0c4d864
4,$3.061.869,13% OFF,9 cuotas de $340.208 con 0% interés,No informacion found,68b09ec6210a34c1d0c4d865
5,$2.748.552,15% OFF,9 cuotas de $305.395 con 0% interés,No informacion found,68b09ec6210a34c1d0c4d866
6,$1.549.900,32% OFF,9 cuotas de $172.211 con 0% interés,"[Con pantalla táctil: No, Conexión wifi y blue...",68b09ec6210a34c1d0c4d867
7,$2.849.900,No discount found,9 cuotas de $316.656 con 0% interés,No informacion found,68b09ec6210a34c1d0c4d868
8,$2.016.630,32% OFF,9 cuotas de $224.070 con 0% interés,"[Capacidad de disco SSD: 512 GB, Capacidad tot...",68b09ec6210a34c1d0c4d869
9,$3.199.900,No discount found,9 cuotas de $355.544 con 0% interés,No informacion found,68b09ec6210a34c1d0c4d86a


## Transform

In [58]:
df_mod = df

In [59]:
df_mod["Precio"] = df_mod["precio"].str.replace(r'[^\d]', '', regex=True).astype(int)
df_mod["Descuento (%)"] = df_mod["descuento"].str.extract(r'(\d+)', expand=False).fillna('0').astype(int)
df_mod["Cuotas"] = df_mod["cuotas"].str.extract(r'(\d+)\s*cuotas', expand=False).fillna('0').astype(int)
df_mod["Valor Cuotas"] = df_mod["cuotas"].str.extract(r"de\s*\$([\d\.]+)", expand=False).fillna('0').str.replace(',', '.').str.replace('.', '').astype(int)
df_mod["Interes (%)"] = df_mod["cuotas"].str.extract(r'Interés de\s*\$?\s*([\d.,]+)', expand=False).fillna('0').str.replace('.', '').str.replace(',', '.').astype(float)
df_mod["Informacion"] = df_mod["informacion"]
df_mod.drop(columns=["precio", "descuento", "cuotas", "informacion"], inplace=True)
df_mod

,_id,Valor Cuotas,Cuotas,Interes (%),Precio,Descuento,Descuento (%),Informacion
0,68b09ec6210a34c1d0c4d861,626656,9,0.0,5639900,16,16,No informacion found
1,68b09ec6210a34c1d0c4d862,224571,9,0.0,2021136,0,0,"[Procesador: Intel Core i5 1235U, Nombre del s..."
2,68b09ec6210a34c1d0c4d863,195983,6,0.0,1175900,0,0,"[Capacidad de disco SSD: 256 GB, Capacidad tot..."
3,68b09ec6210a34c1d0c4d864,171100,9,0.0,1539900,0,0,"[Procesador: Intel Core i3 1215U, Versión del ..."
4,68b09ec6210a34c1d0c4d865,340208,9,0.0,3061869,13,13,No informacion found
5,68b09ec6210a34c1d0c4d866,305395,9,0.0,2748552,15,15,No informacion found
6,68b09ec6210a34c1d0c4d867,172211,9,0.0,1549900,32,32,"[Con pantalla táctil: No, Conexión wifi y blue..."
7,68b09ec6210a34c1d0c4d868,316656,9,0.0,2849900,0,0,No informacion found
8,68b09ec6210a34c1d0c4d869,224070,9,0.0,2016630,32,32,"[Capacidad de disco SSD: 512 GB, Capacidad tot..."
9,68b09ec6210a34c1d0c4d86a,355544,9,0.0,3199900,0,0,No informacion found


## Load

In [19]:
from pymongo import MongoClient
from datetime import datetime
import pprint

In [60]:
client = MongoClient("localhost:27017")

db = client["Meli"]
collection = db["coleccion"]

result = collection.insert_many(df_mod.to_dict('records'))
print("IDs insertados:", result.inserted_ids)

IDs insertados: [ObjectId('68b09ec6210a34c1d0c4d861'), ObjectId('68b09ec6210a34c1d0c4d862'), ObjectId('68b09ec6210a34c1d0c4d863'), ObjectId('68b09ec6210a34c1d0c4d864'), ObjectId('68b09ec6210a34c1d0c4d865'), ObjectId('68b09ec6210a34c1d0c4d866'), ObjectId('68b09ec6210a34c1d0c4d867'), ObjectId('68b09ec6210a34c1d0c4d868'), ObjectId('68b09ec6210a34c1d0c4d869'), ObjectId('68b09ec6210a34c1d0c4d86a'), ObjectId('68b09ec6210a34c1d0c4d86b'), ObjectId('68b09ec6210a34c1d0c4d86c'), ObjectId('68b09ec6210a34c1d0c4d86d'), ObjectId('68b09ec6210a34c1d0c4d86e'), ObjectId('68b09ec6210a34c1d0c4d86f'), ObjectId('68b09ec6210a34c1d0c4d870'), ObjectId('68b09ec6210a34c1d0c4d871'), ObjectId('68b09ec6210a34c1d0c4d872'), ObjectId('68b09ec6210a34c1d0c4d873'), ObjectId('68b09ec6210a34c1d0c4d874')]


In [ ]:
# ¿Cuál es el producto más caro y más barato?
mas_caro = collection.find_one(sort=[("Precio", -1)], projection={"_id":0, "Precio":1, "Informacion":1})
mas_barato = collection.find_one(sort=[("Precio", 1)], projection={"_id":0, "Precio":1, "Informacion":1})

print("Más caro:", mas_caro)
print("Más barato:", mas_barato)

Más caro: {'Precio': 5639900, 'Informacion': 'No informacion found'}
Más barato: {'Precio': 1175900, 'Informacion': ['Capacidad de disco SSD: 256 GB', 'Capacidad total del módulo de memoria RAM: 8 GB', 'Voltaje: 110V', 'GPU: Intel HD Graphics', 'Pantalla LED Full HD', ' Tiene micrófono incorporado', 'Con Bluetooth y Wi-Fi', 'Con 5 puertos USB 2 USB Tipo-A.']}


In [ ]:
# ¿Cuál es el precio promedio de los productos?
consulta = list(collection.aggregate([
    {"$group": {"_id": None, "precio_promedio": {"$avg": "$Precio"}}}
]))
print(consulta)

[{'_id': None, 'precio_promedio': 2591023.25}]


In [ ]:
# ¿Cuantos productos tienen descuento y cuál es el máximo descuento?
consulta = list(collection.aggregate([
    {"$match": {"Descuento (%)": {"$gt": 0}}},
    {"$group": {
        "_id": None,
        "productos_con_descuento": {"$sum": 1},
        "max_descuento": {"$max": "$Descuento (%)"}
    }}
]))
print(consulta)

[{'_id': None, 'productos_con_descuento': 13, 'max_descuento': 33}]


In [ ]:
# Buscar los productos con Intel o SSD dentro de la información
consulta = list(collection.find(
    {"Informacion": {"$regex": "(Intel|SSD)", "$options": "i"}},
    {"_id":0, "Precio":1, "Descuento (%)":1, "Informacion":1}
))
for prod in consulta:
    print(prod)

{'Precio': 2021136, 'Descuento (%)': 0, 'Informacion': ['Procesador: Intel Core i5 1235U', 'Nombre del sistema operativo: FreeDOS', 'Capacidad de disco SSD: 512 GB', 'Capacidad total del módulo de memoria RAM: 24 GB', 'Tarjeta gráfica: Gráficos Intel Iris X', 'Con pantalla táctil: No', 'Resolución de la pantalla: 1920 px x 1080 px', 'Conexión wifi y bluetooth', 'Cuenta con puerto USB y puerto HDMI.']}
{'Precio': 1175900, 'Descuento (%)': 0, 'Informacion': ['Capacidad de disco SSD: 256 GB', 'Capacidad total del módulo de memoria RAM: 8 GB', 'Voltaje: 110V', 'GPU: Intel HD Graphics', 'Pantalla LED Full HD', ' Tiene micrófono incorporado', 'Con Bluetooth y Wi-Fi', 'Con 5 puertos USB 2 USB Tipo-A.']}
{'Precio': 1539900, 'Descuento (%)': 0, 'Informacion': ['Procesador: Intel Core i3 1215U', 'Versión del sistema operativo: 11', 'Edición del sistema operativo: FREE DOS', 'Nombre del sistema operativo: Windows', 'Capacidad de disco SSD: 512 GB', 'Capacidad total del módulo de memoria RAM: 16 G